In [1]:
#! /usr/bin/env python
"""
Compute elevation statistics for the debris-covered areas in each latitude and longitude
"""

import sys
import os
import re
import subprocess
from datetime import datetime, timedelta
import time
import pickle
from collections import OrderedDict

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
from scipy import ndimage
import xarray as xr
from osgeo import gdal, ogr, osr

from pygeotools.lib import malib, warplib, geolib, iolib, timelib

import debrisglobal.globaldebris_input as debris_prms

In [2]:
# Debris cover extent shapefile with statistics
dc_shp = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
dc_shp = dc_shp.sort_values(by=['RGIId'])

print('All DC glaciers:', dc_shp.shape[0], 'All DC Area (km2):', dc_shp.DC_Area_v2.sum() / 1e6)

# Subset by percent debris-covered or debris-covered area
dc_shp_subset = dc_shp[((dc_shp['DC_Area__1'] > debris_prms.dc_percarea_threshold) | 
                        (dc_shp['DC_Area_v2'] / 1e6 > debris_prms.dc_area_threshold))
                        & (dc_shp['Area'] > debris_prms.min_glac_area)].copy()
dc_shp_subset.reset_index(inplace=True, drop=True)

rgino_str_list_subset = [x.split('-')[1] for x in dc_shp_subset.RGIId.values]

print('Subset DC glaciers:', dc_shp_subset.shape[0], 'Subset DC Area (km2):', dc_shp_subset.DC_Area_v2.sum() / 1e6)

dc_shp_subset

All DC glaciers: 6834 All DC Area (km2): 6959.884397
Subset DC glaciers: 1109 Subset DC Area (km2): 6352.559502


,RGIId,GLIMSId,BgnDate,EndDate,CenLon,CenLat,O1Region,O2Region,Area,Zmin,...,Name,DC_Area,DC_BgnDate,DC_EndDate,DC_CTSmean,DC_Area_%,area_singl,DC_Area_v2,DC_Area__1,geometry
0,RGI60-01.00006,G213756E63571N,20090703,-9999999,-146.269039,63.565440,1,2,10.470,1201,...,None,1268100,2013,2017,50.395919,12.11,50438,1194194,11.41,"MULTIPOLYGON (((-146.21212 63.58992, -146.2115..."
1,RGI60-01.00013,G213316E63499N,20090703,-9999999,-146.783771,63.548672,1,2,209.630,823,...,Susitna Glacier,37403100,2013,2017,52.933578,17.84,32298677,38937084,18.57,"MULTIPOLYGON (((-146.69834 63.51530, -146.6965..."
2,RGI60-01.00027,G213737E63535N,20090703,-9999999,-146.235312,63.539164,1,2,13.290,1073,...,McGinnis Glacier,1243800,2013,2017,52.228899,9.36,129699,1116507,8.40,"MULTIPOLYGON (((-146.20847 63.54709, -146.2078..."
3,RGI60-01.00033,G213128E63680N,20090703,-9999999,-146.870135,63.680161,1,2,4.604,1109,...,None,940500,2013,2017,63.881376,20.43,27022,897454,19.49,"MULTIPOLYGON (((-146.87661 63.69865, -146.8766..."
4,RGI60-01.00035,G212558E63648N,20090703,-9999999,-147.438831,63.649559,1,2,36.349,1274,...,None,1401300,2013,2017,42.695614,3.86,27021,1237798,3.41,"MULTIPOLYGON (((-147.55037 63.61418, -147.5503..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1104,RGI60-01.26861,G225268E59454N,20050811,-9999999,-134.734662,59.455168,1,6,3.545,1404,...,None,430200,2013,2017,28.079949,12.14,23419,369293,10.42,"MULTIPOLYGON (((-134.72630 59.44781, -134.7252..."
1105,RGI60-01.27101,G216527E60710N,20100919,-9999999,-143.419543,60.736680,1,5,80.219,492,...,North Fork Lobe Bremner Glacier,15471900,2013,2017,44.530354,19.29,245801,15504591,19.33,"MULTIPOLYGON (((-143.32822 60.84214, -143.3265..."
1106,RGI60-01.27103,G225914E58943N,20050811,-9999999,-134.072577,58.948370,1,6,86.656,873,...,None,1487700,2013,2017,27.500816,1.72,56741,1312248,1.51,"MULTIPOLYGON (((-134.05165 58.99606, -134.0511..."
1107,RGI60-01.27105,G227608E57164N,20040810,-9999999,-132.400219,57.153059,1,6,131.574,507,...,None,21989700,2013,2017,21.858476,16.71,50409,21268145,16.16,"MULTIPOLYGON (((-132.29926 57.14057, -132.2992..."


In [3]:
# Select glaciers using RGI and find unique latlons
#  (Scherler DC shapefiles do not have same CenLat and CenLon for some reason)
main_glac_rgi_subset = debris_prms.selectglaciersrgitable(rgino_str_list_subset)
main_glac_rgi_subset['CenLon_360'] = main_glac_rgi_subset['CenLon']
main_glac_rgi_subset.loc[main_glac_rgi_subset['CenLon_360'] < 0, 'CenLon_360'] = (
    360 + main_glac_rgi_subset.loc[main_glac_rgi_subset['CenLon_360'] < 0, 'CenLon_360'])

# Load met data and find nearest latlon indices
ds = xr.open_dataset(debris_prms.metdata_fp + '../' + debris_prms.metdata_elev_fn)
#  argmin() finds the minimum distance between the glacier lat/lon and the GCM pixel
lat_nearidx = (np.abs(main_glac_rgi_subset['CenLat'].values[:,np.newaxis] - 
                      ds['latitude'][:].values).argmin(axis=1))
lon_nearidx = (np.abs(main_glac_rgi_subset['CenLon_360'].values[:,np.newaxis] - 
                      ds['longitude'][:].values).argmin(axis=1))
latlon_nearidx = list(zip(lat_nearidx, lon_nearidx))
latlon_nearidx_unique = sorted(list(set(latlon_nearidx)))
main_glac_rgi_subset['latlon_nearidx'] = latlon_nearidx
latlon_unique_dict = dict(zip(latlon_nearidx_unique,np.arange(0,len(latlon_nearidx_unique))))
latlon_unique_dict_reversed = dict(zip(np.arange(0,len(latlon_nearidx_unique)),latlon_nearidx_unique))
main_glac_rgi_subset['latlon_unique_no'] = main_glac_rgi_subset['latlon_nearidx'].map(latlon_unique_dict)

print('unique lat/lons:', len(np.unique(main_glac_rgi_subset['latlon_unique_no'])), '\n\n')

# Delete me
latlon_nearidx_unique_v1 = latlon_nearidx_unique.copy()

lat_list = np.array([ds.latitude[x[0]].values for x in latlon_nearidx_unique])
lon_list = np.array([ds.longitude[x[1]].values for x in latlon_nearidx_unique])
latlon_list = list(tuple(zip(list(lat_list), list(lon_list))))

# Pickle unique lat/lons that will be used for melt model
with open(debris_prms.latlon_unique_fp + debris_prms.latlon_unique_dict[debris_prms.roi], 'wb') as f:
    pickle.dump(latlon_list, f)

1109 glaciers in region 1 are included in this model run: ['00006', '00013', '00027', '00033', '00035', '00037', '00038', '00040', '00041', '00042', '00044', '00045', '00046', '00140', '00148', '00187', '00242', '00312', '00336', '00348', '00351', '00399', '00409', '00426', '00434', '00436', '00537', '00544', '00556', '00557', '00558', '00560', '00561', '00565', '00566', '00569', '00570', '00571', '00572', '00574', '00576', '00578', '00579', '00581', '00582', '00584', '00600', '00660', '00670', '00675'] and more
This study is focusing on 1109 glaciers in region [1]
unique lat/lons: 457 




In [4]:
# ===== LOAD GLACIERS WITH DATA =====
main_glac_rgi_subset['mb_fullfn'] = np.nan
mb_binned_fp = debris_prms.mb_binned_fp

regions_str = [str(x).zfill(2) for x in debris_prms.roi_rgidict[debris_prms.roi]]

mb_fns = []
mb_rgiids = []
for i in os.listdir(mb_binned_fp):
    if i.endswith('_mb_bins.csv') and i.split('_')[0].split('.')[0].zfill(2) in regions_str:
        mb_fns.append(mb_binned_fp + i)
        rgiid_raw = i.split('_')[0]
        rgiid = 'RGI60-' + rgiid_raw.split('.')[0].zfill(2) + '.' + rgiid_raw.split('.')[1]
        mb_rgiids.append(rgiid)
mb_rgiids = sorted(mb_rgiids)
mb_fn_df = pd.DataFrame(np.zeros((len(mb_fns),2)), columns=['RGIId', 'mb_fn'])
mb_fn_df['RGIId'] = mb_rgiids
mb_fn_df['mb_fullfn'] = mb_fns

# Find glaciers that are debris-covered
mb_dc_rgiid = [value for value in list(mb_fn_df.RGIId.values) if value in list(main_glac_rgi_subset.RGIId.values)]
mb_fn_df_dc = mb_fn_df[mb_fn_df['RGIId'].isin(mb_dc_rgiid)]
mb_fn_df_dc = mb_fn_df_dc.sort_values('RGIId')

print('Debris-covered glaciers:', mb_fn_df_dc.shape[0], '\n\n')

mb_fn_dict = dict(zip(mb_fn_df_dc['RGIId'].values, mb_fn_df_dc['mb_fullfn'].values))

main_glac_rgi_subset['mb_fullfn'] = main_glac_rgi_subset.RGIId.map(mb_fn_dict)

Debris-covered glaciers: 1109 




In [5]:
# ===== SELECT GLACIERS WITH DATA ====
main_glac_rgi_wobs = main_glac_rgi_subset.dropna(subset=['mb_fullfn']).copy()
# print('subset wdata length:', main_glac_rgi_wobs.shape)
main_glac_rgi_wobs.reset_index(inplace=True, drop=True)

# Update the latlon unique pickle files
latlon_nearidx_unique = sorted(list(set(main_glac_rgi_wobs['latlon_nearidx'].values)))
latlon_unique_dict = dict(zip(latlon_nearidx_unique,np.arange(0,len(latlon_nearidx_unique))))
latlon_unique_dict_reversed = dict(zip(np.arange(0,len(latlon_nearidx_unique)),latlon_nearidx_unique))
main_glac_rgi_wobs['latlon_unique_no'] = main_glac_rgi_wobs['latlon_nearidx'].map(latlon_unique_dict)

print('unique lat/lons:', len(np.unique(main_glac_rgi_wobs['latlon_unique_no'])), '\n\n')

lat_list = np.array([ds.latitude[x[0]].values for x in latlon_nearidx_unique])
lon_list = np.array([ds.longitude[x[1]].values for x in latlon_nearidx_unique])
latlon_list = list(tuple(zip(list(lat_list), list(lon_list))))

# Pickle unique lat/lons that will be used for melt model
with open(debris_prms.latlon_unique_fp + debris_prms.latlon_unique_dict[debris_prms.roi], 'wb') as f:
    pickle.dump(latlon_list, f)
    
main_glac_rgi_wobs

unique lat/lons: 457 




,O1Index,RGIId,CenLon,CenLat,O1Region,O2Region,Area,Zmin,Zmax,Zmed,...,TermType,Surging,RefDate,glacno,rgino_str,RGIId_float,CenLon_360,latlon_nearidx,latlon_unique_no,mb_fullfn
0,5,RGI60-01.00006,-146.244000,63.571000,1,2,10.470,1201,3547,1740,...,0,9,20090703,6,01.00006,1.00006,213.756000,"(106, 855)",11,/Users/davidrounce/Documents/Dave_Rounce/Debri...
1,12,RGI60-01.00013,-146.684082,63.499329,1,2,209.630,823,4003,1848,...,0,3,20090703,13,01.00013,1.00013,213.315918,"(106, 853)",9,/Users/davidrounce/Documents/Dave_Rounce/Debri...
2,26,RGI60-01.00027,-146.262817,63.535065,1,2,13.290,1073,2985,1742,...,0,3,20090703,27,01.00027,1.00027,213.737183,"(106, 855)",11,/Users/davidrounce/Documents/Dave_Rounce/Debri...
3,32,RGI60-01.00033,-146.872000,63.680000,1,2,4.604,1109,3160,1718,...,0,9,20090703,33,01.00033,1.00033,213.128000,"(105, 853)",4,/Users/davidrounce/Documents/Dave_Rounce/Debri...
4,34,RGI60-01.00035,-147.442000,63.648000,1,2,36.349,1274,2928,1913,...,0,9,20090703,35,01.00035,1.00035,212.558000,"(105, 850)",1,/Users/davidrounce/Documents/Dave_Rounce/Debri...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1104,26856,RGI60-01.26861,-134.732000,59.454000,1,6,3.545,1404,1879,1506,...,0,9,20050811,26861,01.26861,1.26861,225.268000,"(122, 901)",293,/Users/davidrounce/Documents/Dave_Rounce/Debri...
1105,27096,RGI60-01.27101,-143.473236,60.709583,1,5,80.219,492,3182,1403,...,0,9,20100919,27101,01.27101,1.27101,216.526764,"(117, 866)",182,/Users/davidrounce/Documents/Dave_Rounce/Debri...
1106,27098,RGI60-01.27103,-134.086000,58.943000,1,6,86.656,873,2236,1768,...,0,9,20050811,27103,01.27103,1.27103,225.914000,"(124, 904)",326,/Users/davidrounce/Documents/Dave_Rounce/Debri...
1107,27100,RGI60-01.27105,-132.392000,57.164000,1,6,131.574,507,2837,1219,...,0,9,20040810,27105,01.27105,1.27105,227.608000,"(131, 910)",397,/Users/davidrounce/Documents/Dave_Rounce/Debri...


In [6]:
# ===== DEBRIS ELEVATION STATS ====================================================================================
# CALCULATE DEBRIS ELEVATION STATS FOR GLACIERS WITH DATA FOR EACH UNIQUE LAT/LON
elev_stats_latlon_dict = {}
latlon_list_updated = []
rgiid_4cal = []
for nlatlon, latlon_unique in enumerate(np.unique(main_glac_rgi_wobs.latlon_unique_no)):
# for nlatlon, latlon_unique in enumerate([np.unique(main_glac_rgi_wobs.latlon_unique_no)[0]]):

    main_glac_rgi_subset = main_glac_rgi_wobs[main_glac_rgi_wobs['latlon_unique_no'] == latlon_unique]

    # Debris elevation stats should be done by lat/lon
    df_all = None
    elev_list_all = []
    df_idx_count = 0
    count_width_passes = 0
    for nglac, glac_fullfn in enumerate(main_glac_rgi_subset.mb_fullfn.values):
#     for nglac, glac_fn in enumerate([main_glac_rgi_subset.mb_fullfn.values[0]]):

        glac_str_noleadzero = glac_fullfn.split('/')[-1].split('_')[0]
        rgiid = 'RGI60-' + glac_str_noleadzero.split('.')[0].zfill(2) + '.' + glac_str_noleadzero.split('.')[1]

        df_raw = pd.read_csv(glac_fullfn)
        df = df_raw.dropna(subset=['mb_bin_mean_mwea'])
        df_debris = df[(df['vm_med'] < debris_prms.vel_threshold) & 
                       (df['dc_bin_area_perc'] > debris_prms.debrisperc_threshold)
                       & (df['dc_bin_count_valid'] > 0)]

        df_idx = df_debris.index.values
        df_idx_count += len(df_idx)
        
        # Widths
        widths_fp = debris_prms.oggm_fp + 'widths/' + 'RGI60-' + rgiid.split('-')[1].split('.')[0] + '/'
        widths_fn = rgiid + '_widths_m.csv'
        try:
            widths_df = pd.read_csv(widths_fp + widths_fn)
            h = widths_df['elev'].values
            widths_m = widths_df['width_m'].values
        except:
            widths_df = None

        if len(df_idx) > 0 and widths_df is not None:
            # only work with terminus
            df_idx_dif = list(df_idx[1:] - df_idx[:-1])
            if np.sum(df_idx_dif) == len(df_idx)-1:
                df_idx_nojump = df_idx
            else:
                idx_jumpinbins = df_idx_dif.index(next(filter(lambda x: x>1, df_idx_dif)))
                df_idx_nojump = df_idx[0:idx_jumpinbins+1]

            df_debris_nojump = df_debris.loc[df_idx_nojump,:]
            df_debris_nojump.reset_index(inplace=True, drop=True)
            
            # Median width to ensure terminus velocities can be estimated
            width_median = np.median(widths_m[np.where(h < df_debris_nojump['bin_center_elev_m'].max())[0]])
            
            if width_median > debris_prms.width_min_dict[debris_prms.roi]:
                for nelev, elev in enumerate(list(df_debris_nojump['bin_center_elev_m'].values)):
                    elev_list_single = list(np.repeat(elev, df_debris_nojump.loc[nelev,'dc_bin_count_valid']))
                    elev_list_all.extend(elev_list_single)
                count_width_passes += 1
                
            rgiid_4cal.append(rgiid.split('-')[1])
        
    if df_idx_count > 0 and count_width_passes > 0:
        dc_zmean = np.mean(elev_list_all)
        dc_zstd = np.std(elev_list_all)
        dc_zmed = malib.fast_median(elev_list_all)
        dc_zmad = malib.mad(elev_list_all)

        lat_deg = float(ds.latitude[latlon_unique_dict_reversed[latlon_unique][0]].values)
        lon_deg = float(ds.longitude[latlon_unique_dict_reversed[latlon_unique][1]].values)
        elev_stats_latlon_dict[lat_deg,lon_deg] = [dc_zmean, dc_zstd, dc_zmed, dc_zmad]
        latlon_list_updated.append((lat_deg, lon_deg))
        
print('unique lat/lons updated:', len(latlon_list_updated))
# Update pickle of unique lat/lons that will be used for melt model
with open(debris_prms.latlon_unique_fp + debris_prms.latlon_unique_dict[debris_prms.roi], 'wb') as f:
    pickle.dump(latlon_list_updated, f)

/Users/davidrounce/anaconda3/envs/debris_thickness_global/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/davidrounce/anaconda3/envs/debris_thickness_global/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


unique lat/lons updated: 396


In [7]:
# Statistics of data coverage
rgiid_4cal = sorted(rgiid_4cal)
main_glac_rgi_4cal = debris_prms.selectglaciersrgitable(rgiid_4cal)
dc_area_dict = dict(zip(dc_shp.RGIId.values, dc_shp.DC_Area_v2.values))
main_glac_rgi_4cal['DC_Area_v2'] = main_glac_rgi_4cal.RGIId.map(dc_area_dict)
print('\nDC glaciers (used for cal):', main_glac_rgi_4cal.shape[0], 
      'DC Area (used for cal, km2):', main_glac_rgi_4cal.DC_Area_v2.sum() / 1e6)

1010 glaciers in region 1 are included in this model run: ['00006', '00013', '00027', '00033', '00037', '00038', '00040', '00041', '00042', '00044', '00045', '00046', '00140', '00148', '00187', '00242', '00312', '00348', '00351', '00399', '00409', '00426', '00434', '00436', '00537', '00544', '00556', '00557', '00558', '00560', '00561', '00565', '00566', '00569', '00570', '00571', '00572', '00574', '00576', '00578', '00579', '00581', '00582', '00584', '00600', '00660', '00670', '00675', '00732', '00739'] and more
This study is focusing on 1010 glaciers in region [1]

DC glaciers (used for cal): 1010 DC Area (used for cal, km2): 5757.058539


In [8]:
# ===== ADD DEBRIS ELEVATION STATS TO MET DATA ======
overwrite_dc_stats = True
for nlatlon, latlon in enumerate(latlon_list_updated):
# for nlatlon, latlon in enumerate([latlon_list_updated[0]]):
    
    lat_deg = latlon[0]
    lon_deg = latlon[1]
    
    print(nlatlon, lat_deg, lon_deg)
    
    if lat_deg < 0:
        lat_str = 'S-'
    else:
        lat_str = 'N-' 

    # ===== Meteorological data =====
    metdata_fn = debris_prms.metdata_fn_sample.replace(
        'XXXX', str(int(np.abs(lat_deg)*100)) + lat_str + str(int(lon_deg*100)) + 'E-')
    
    ds = xr.open_dataset(debris_prms.metdata_fp + metdata_fn)    
#     print('  ', ds.dc_zmean.values, elev_stats_latlon_dict[latlon][0])
    if 'dc_zmean' not in list(ds.keys()) or overwrite_dc_stats:
        # Add stats
        ds['dc_zmean'] = elev_stats_latlon_dict[latlon][0]
        ds['dc_zmean'].attrs = {'units':'m a.s.l.', 'long_name':'Mean debris cover elevation', 
                                'comment':'converted from debris cover with data that will be used for subdebris melt inversion'}
        ds['dc_zstd'] = elev_stats_latlon_dict[latlon][1]
        ds['dc_zstd'].attrs = {'units':'m a.s.l.', 'long_name':'Standard deviation of debris cover elevation', 
                               'comment':'converted from debris cover with data that will be used for subdebris melt inversion'}
        ds['dc_zmed'] = elev_stats_latlon_dict[latlon][2]
        ds['dc_zmed'].attrs = {'units':'m a.s.l.', 'long_name':'Median debris cover elevation', 
                               'comment':'converted from debris cover with data that will be used for subdebris melt inversion'}
        ds['dc_zmad'] = elev_stats_latlon_dict[latlon][3]
        ds['dc_zmad'].attrs = {'units':'m a.s.l.', 'long_name':'Median absolute deviation of debris cover elevation', 
                               'comment':'converted from debris cover with data that will be used for subdebris melt inversion'}

        try:
            ds.close()
        except:
            continue
            
        # Export updated dataset
        ds.to_netcdf(debris_prms.metdata_fp + metdata_fn, mode='a')
    else:
        print(lat_deg, lon_deg, 'exists')

0 63.75 212.25
1 63.75 212.75
2 63.75 213.0
3 63.75 213.25
4 63.5 210.5
5 63.5 212.5
6 63.5 213.5
7 63.5 213.75
8 63.5 214.25
9 63.5 214.5
10 63.5 214.75
11 63.5 215.25
12 63.25 209.0
13 63.25 209.5
14 63.25 210.0
15 63.25 214.5
16 63.25 214.75
17 63.25 215.0
18 63.25 215.25
19 63.25 215.5
20 63.25 215.75
21 63.0 208.5
22 63.0 208.75
23 63.0 209.25
24 63.0 209.5
25 62.75 208.0
26 62.75 208.25
27 62.75 208.5
28 62.75 208.75
29 62.75 209.0
30 62.5 207.25
31 62.5 207.5
32 62.5 207.75
33 62.25 215.25
34 62.25 215.75
35 62.25 216.0
36 62.25 217.0
37 62.25 217.25
38 62.25 217.5
39 62.25 218.5
40 62.0 207.0
41 62.0 207.5
42 62.0 211.5
43 62.0 215.25
44 62.0 215.5
45 62.0 215.75
46 62.0 216.0
47 62.0 216.25
48 62.0 216.5
49 62.0 216.75
50 62.0 217.0
51 62.0 217.25
52 62.0 217.5
53 62.0 217.75
54 62.0 218.5
55 62.0 218.75
56 61.75 205.75
57 61.75 206.0
58 61.75 206.25
59 61.75 207.0
60 61.75 207.25
61 61.75 207.5
62 61.75 208.0
63 61.75 211.75
64 61.75 212.5
65 61.75 216.25
66 61.75 216.5
67 61

In [11]:
# ==== OLD FILE OF LOADIING MULTIPLE DATASETS =====
# # ===== LOAD GLACIERS WITH LARSEN DATA =====
# dc_shp_subset['larsen_fullfn'] = np.nan
# larsen_fullfn_dict = {}
# if 'larsen' in input.mb_datasets:
#     mb_summary = pd.read_csv(input.larsen_fp + input.larsen_fn)
    
#     # Find glaciers that are debris-covered
#     larsen_dc_rgiid = [value for value in list(mb_summary.RGIId.values) 
#                        if value in list(dc_shp_subset.RGIId.values)]

#     mb_summary_dc = mb_summary[mb_summary['RGIId'].isin(larsen_dc_rgiid)]
#     mb_summary_dc = mb_summary_dc.sort_values('RGIId')
#     mb_summary_dc.reset_index(inplace=True, drop=True)
#     mb_summary_dc.loc[mb_summary_dc['name'] == 'Maclaren', 'name'] = 'MacLaren'
#     mb_summary_dc.loc[mb_summary_dc['name'] == 'Tlikakila Fork', 'name'] = 'TlikakilaGlacierFork'
#     mb_summary_dc.loc[mb_summary_dc['name'] == 'Tlikakila N. Fork', 'name'] = 'TlikakilaNorthFork'
#     mb_summary_dc['larsen_fullfn'] = np.nan
    
#     for n, glac_name in enumerate(mb_summary_dc.name.values):
# #     for n, glac_name in enumerate([mb_summary_dc.name.values[47]]):
# #         print(n, glac_name)
            
#         glac_name = glac_name.replace(' ', '')
#         glac_fns = []
#         start_yr = []
#         end_yr = []
#         for i in os.listdir(input.larsen_binned_fp):
#             if i.startswith(glac_name):
#                 glac_fns.append(i)
#                 start_yr.append(i.split('.')[1][0:4])
#                 end_yr.append(i.split('.')[2][0:4])
                
#         if len(glac_fns) > 0:
#             yr_dif = np.array(end_yr).astype(int) - np.array(start_yr).astype(int)
#             mb_fn = glac_fns[np.where(yr_dif == yr_dif.max())[0][0]]
            
#             # ===== Process Larsen dataset =====
#             larsen_data_raw = np.genfromtxt(input.larsen_binned_fp + mb_fn, skip_header=3)
#             larsen_data_header = ['E', 'DZ', 'DZ25', 'DZ75', 'AAD', 'MassChange', 'MassBal', 'NumData']
#             larsen_data = pd.DataFrame(larsen_data_raw, columns=larsen_data_header)
#             larsen_data['std from DZ25'] = np.absolute(larsen_data['DZ'] - larsen_data['DZ25']) / 0.67
#             larsen_data['std from DZ75'] = np.absolute(larsen_data['DZ'] - larsen_data['DZ75']) / 0.67
#             larsen_data[' dhdt_bin_std_ma'] = (larsen_data['std from DZ25'] + larsen_data['std from DZ75']) / 2
#             larsen_data[' mb_bin_std_mwea'] = larsen_data[' dhdt_bin_std_ma'] * 900 / 1000
#             larsen_data['AAD'] = larsen_data['AAD'] / 1e6
#             larsen_data['startyear'] = int(mb_fn.split('.')[1][0:4])
#             larsen_data['endyear'] = int(mb_fn.split('.')[2][0:4])
#             larsen_data = larsen_data.rename({'E': '# bin_center_elev_m',
#                                               'DZ': ' dhdt_bin_mean_ma',
#                                               'MassBal': ' mb_bin_mean_mwea',
#                                               'AAD': ' z1_bin_area_valid_km2',
#                                              }, axis='columns')
#             new_fn = mb_summary_dc.loc[n,'RGIId'].split('-')[1][1:] + '_larsen_mb_bins.csv'
#             larsen_data.to_csv(input.larsen_binned_fp + new_fn, index=False)
            
#             mb_summary_dc.loc[n, 'larsen_fullfn'] = input.larsen_binned_fp + new_fn
            
#         else:
#             print(n, glac_name, 'has no file\n')

#     mb_summary_dc.dropna(subset=['larsen_fullfn'], inplace=True)
#     mb_summary_dc.reset_index(inplace=True, drop=True)
    
#     print('Larsen debris-covered glaciers:', mb_summary_dc.shape[0], '\n\n')
    
#     larsen_fullfn_dict = dict(zip(mb_summary_dc['RGIId'].values, mb_summary_dc['larsen_fullfn'].values))
# #     print(larsen_fullfn_dict)
#     dc_shp_subset['larsen_fullfn'] = dc_shp_subset.RGIId.map(larsen_fullfn_dict)

# # ===== LOAD GLACIERS WITH BRAUN DATA =====
# dc_shp_subset['braun_fullfn'] = np.nan
# braun_fullfn_dict = {}
# if 'braun' in input.mb_datasets:
#     mb_binned_fp = input.main_directory + '/../mb_data/Braun/binned_data/'
# #     mb_binned_fp = input.mb_binned_fp
    
#     mb_fns = []
#     braun_rgiids = []
#     for i in os.listdir(mb_binned_fp):
#         if i.endswith('_mb_bins.csv'):
#             mb_fns.append(mb_binned_fp + i)
#             rgiid_raw = i.split('_')[0]
#             rgiid = 'RGI60-' + rgiid_raw.split('.')[0].zfill(2) + '.' + rgiid_raw.split('.')[1]
#             braun_rgiids.append(rgiid)
#     braun_fn_df = pd.DataFrame(np.zeros((len(mb_fns),2)), columns=['RGIId', 'braun_fn'])
#     braun_fn_df['RGIId'] = braun_rgiids
#     braun_fn_df['braun_fullfn'] = mb_fns
    
#     # Find glaciers that are debris-covered
#     braun_dc_rgiid = [value for value in list(braun_fn_df.RGIId.values) 
#                        if value in list(dc_shp_subset.RGIId.values)]
#     braun_fn_df_dc = braun_fn_df[braun_fn_df['RGIId'].isin(braun_dc_rgiid)]
#     braun_fn_df_dc = braun_fn_df_dc.sort_values('RGIId')
    
#     print('Braun debris-covered glaciers:', braun_fn_df_dc.shape[0], '\n\n')
    
#     braun_fullfn_dict = dict(zip(braun_fn_df_dc['RGIId'].values, braun_fn_df_dc['braun_fullfn'].values))
    
#     dc_shp_subset['braun_fullfn'] = dc_shp_subset.RGIId.map(braun_fullfn_dict)

# # ===== LOAD GLACIERS WITH SHEAN DATA =====
# dc_shp_subset['shean_fullfn'] = np.nan
# shean_fullfn_dict = {}
# if 'shean' in input.mb_datasets:
# #     mb_binned_fp = input.main_directory + '/../mb_data/Shean_2019_0213/mb_combined_20190213_nmad_bins/'
#     mb_binned_fp = input.mb_binned_fp
    
#     mb_fns = []
#     rgiids = []
#     for i in os.listdir(mb_binned_fp):
#         if i.endswith('_mb_bins.csv'):
#             mb_fns.append(mb_binned_fp + i)
#             rgiid_raw = i.split('_')[0]
#             rgiid = 'RGI60-' + rgiid_raw.split('.')[0].zfill(2) + '.' + rgiid_raw.split('.')[1]
#             rgiids.append(rgiid)
#     mb_fn_df = pd.DataFrame(np.zeros((len(mb_fns),2)), columns=['RGIId', 'mb_fn'])
#     mb_fn_df['RGIId'] = rgiids
#     mb_fn_df['mb_fullfn'] = mb_fns
    
#     # Find glaciers that are debris-covered
#     mb_dc_rgiid = [value for value in list(mb_fn_df.RGIId.values) 
#                    if value in list(dc_shp_subset.RGIId.values)]
#     mb_fn_df_dc = mb_fn_df[mb_fn_df['RGIId'].isin(mb_dc_rgiid)]
#     mb_fn_df_dc = mb_fn_df_dc.sort_values('RGIId')
    
#     print('shean debris-covered glaciers:', mb_fn_df_dc.shape[0], '\n\n')
    
#     shean_fullfn_dict = dict(zip(mb_fn_df_dc['RGIId'].values, mb_fn_df_dc['mb_fullfn'].values))
# #     print(shea_fullfn_dict)
#     dc_shp_subset['shean_fullfn'] = dc_shp_subset.RGIId.map(shean_fullfn_dict)

# # Merge dictionaries together
# mb_fn_dict = dict(list(larsen_fullfn_dict.items()) + list(braun_fullfn_dict.items()) + 
#                   list(shean_fullfn_dict.items()))